<a href="https://colab.research.google.com/github/ShinJunYoung/NYC_taxi_trip/blob/main/termProject_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

앞선 파일에서 data curation과 data inspection 단계를 거친 이후의 파일이다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.cluster import k_means
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

#추가 경고 출력 안함
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def preprocessing(df):
    df = df.dropna()

    for col in list(df.select_dtypes(include='number').columns):
        df[col] = df[col].apply(lambda x: abs(x) if x < 0 else x)

    # print(df.head)
    df['fare'] = df['total_amount'] - df['tip_amount']
    print(df.head(10))
    print(df['fare'])
    
    print(df.columns)
    print(df.info())

    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y/%m/%d %H:%M:%S')
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y/%m/%d %H:%M:%S')

    df1 = df[(df['tpep_pickup_datetime'] > '2022-02-01 00:00:00') 
            & (df['tpep_pickup_datetime'] < '2022-02-20 23:59:59')]
    df2 = df[(df['tpep_pickup_datetime'] > '2022-02-21 00:00:00') 
            & (df['tpep_pickup_datetime'] < '2022-02-28 23:59:59')]

    print(df1.tpep_pickup_datetime.describe())
    print(df2.tpep_pickup_datetime.describe())

    print(df.tpep_pickup_datetime.describe())

    df['date'] = df['tpep_pickup_datetime'].dt.date
    df['day'] = df['tpep_pickup_datetime'].dt.day
    df['weekday'] = df['tpep_pickup_datetime'].dt.weekday
    

    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'] / np.timedelta64(1, 'm')
    df = df[df['duration'] > 0]
    df[df['duration'] > 180].sort_values('duration', ascending=False)


    df[df['trip_distance'] == 0].sort_values('duration', ascending=False)
    df[(df['trip_distance'] == 0) & (df['duration'] == 1)].sort_values('total_amount', ascending=False)

    # drop no payment (2.5 is default fare)
    df = df[df['fare_amount'] > 2.5]
    #drop zero distance
    df = df[df['trip_distance'] > 0]
    # drop extra
    df = df[df['extra'] < 5]
    df = df[df['passenger_count']>0]
    df = df[df['RatecodeID']<=6]
    df = df[df['weekday']>0]
    
    df['VendorID'] = df['VendorID'].astype('category')
    df['RatecodeID'] = df['RatecodeID'].astype('category')
    df['PULocationID'] = df['PULocationID'].astype('category')
    df['DOLocationID'] = df['DOLocationID'].astype('category')
    df['payment_type'] = df['payment_type'].astype('category')
    
    le = LabelEncoder()
    df['store_and_fwd_flag'] = le.fit_transform(df['store_and_fwd_flag'])


    df = df.drop(['VendorID', 'tpep_dropoff_datetime','tpep_pickup_datetime',
                  'RatecodeID','store_and_fwd_flag','passenger_count',
                       'payment_type','fare_amount','extra','total_amount',
                       'improvement_surcharge','congestion_surcharge',
                       'mta_tax','tip_amount','tolls_amount','date','day'],axis=1)
    
    print(df.head)
    df.to_csv('result.csv')
    
    return df

In [ ]:
def splitData_Stand(df): 
    target = df['fare']    
    data = df.drop(['fare'],axis=1)
    
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    xtrain, xtest , ytrain , ytest = train_test_split(data , target , test_size =0.2 , shuffle = True)
    
    return xtrain, xtest , ytrain , ytest

In [ ]:
def splitData_MinMax(df): 
    target = df['fare']    
    data = df.drop(['fare'],axis=1)
    
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    
    xtrain, xtest , ytrain , ytest = train_test_split(data , target , test_size =0.2 , shuffle = True)
    
    return xtrain, xtest , ytrain , ytest

In [ ]:
from sklearn.preprocessing import RobustScaler
def splitData_Robust(df): 
    target = df['fare']    
    data = df.drop(['fare'],axis=1)
    
    scaler = RobustScaler()
    print(scaler.center_)
    print(scaler.scale_)
    data = scaler.fit_transform(data)

    xtrain, xtest , ytrain , ytest = train_test_split(data , target , test_size =0.2 , shuffle = True)
    
    return xtrain, xtest , ytrain , ytest

In [ ]:
def LinearReg(xtrain, xtest , ytrain , ytest) :
    grid_param = {
        'fit_intercept':[True,False], 
        'normalize':[True,False], 
        'copy_X':[True, False]
    }
    model = LinearRegression()
    grid_search = GridSearchCV(model , param_grid= grid_param, return_train_score=True)
    grid_search.fit(xtrain,ytrain)
    
    #print best parameter setting and best score
    print(grid_search.best_params_)
    print(grid_search.best_score_)

    #train and test score
    print(grid_search.score(xtrain, ytrain)) 
    print(grid_search.score(xtest, ytest))
    df = pd.DataFrame(grid_search.cv_results_)
    df = df.sort_values(by=['mean_test_score', 'mean_train_score'], ascending=False)
    print(df[['params', 'mean_train_score', 'mean_test_score']])

In [ ]:
def DecisionTreeReg(xtrain, xtest , ytrain , ytest):
    grid_param = {'criterion':['squared_error'], 'max_depth':[None,2,3,4,5,6]}
    
    model = DecisionTreeRegressor()
    grid_search = GridSearchCV(model , param_grid= grid_param, return_train_score=True)
    grid_search.fit(xtrain,ytrain)
    
    #print best parameter setting and best score
    print(grid_search.best_params_)
    print(grid_search.best_score_)

    #train and test score
    print(grid_search.score(xtrain, ytrain)) 
    print(grid_search.score(xtest, ytest))
    
    df = pd.DataFrame(grid_search.cv_results_)
    df = df.sort_values(by=['mean_test_score', 'mean_train_score'], ascending=False)
    print(df[['params', 'mean_train_score', 'mean_test_score']])     

In [ ]:
def KNeighborsCla(xtrain, xtest , ytrain , ytest):
    ytrain = ytrain.astype({'fare': 'int'})
    # 모델 선언
    knn = KNeighborsClassifier(n_neighbors = 3)
    knn.fit(xtrain,ytrain)

    score = cross_val_score(knn, xtrain,ytrain, scoring = 'accuracy', cv=5)
    # print each cv score (accuracy) and average them 
    print(score)

In [ ]:
def training(xtrain, xtest , ytrain , ytest):
    
    LinearReg(xtrain, xtest , ytrain , ytest)
    DecisionTreeReg(xtrain, xtest , ytrain , ytest)
    KNeighborsCla(xtrain, xtest , ytrain , ytest)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/colab/yellow_tripdata_2022-02.csv')
df = preprocessing(df)
xtrain, xtest , ytrain , ytest = splitData_Stand(df)
training(xtrain, xtest , ytrain , ytest)
    

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-02-01 00:06:58   2022-02-01 00:19:24              1.0   
1         1  2022-02-01 00:38:22   2022-02-01 00:55:55              1.0   
2         1  2022-02-01 00:03:20   2022-02-01 00:26:59              1.0   
3         2  2022-02-01 00:08:00   2022-02-01 00:28:05              1.0   
4         2  2022-02-01 00:06:48   2022-02-01 00:33:07              1.0   
5         1  2022-02-01 00:57:23   2022-02-01 01:07:31              2.0   
6         1  2022-02-01 00:34:17   2022-02-01 01:01:32              1.0   
7         2  2022-02-01 00:03:26   2022-02-01 00:07:30              1.0   
8         2  2022-02-01 00:13:31   2022-02-01 00:31:41              1.0   
9         2  2022-02-01 00:52:11   2022-02-01 01:02:48              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           5.40         1.0                  N           138           252   
1           6.40

In [ ]:
data_mM=pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/taxidata/yellow_tripdata_2022-02.parquet')
data_mM = preprocessing(data_mM)
xtrain_mM, xtest_mM , ytrain_mM , ytest_mM = splitData_MinMax(data_mM)
training(xtrain_mM, xtest_mM , ytrain_mM , ytest_mM)

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-02-01 00:06:58   2022-02-01 00:19:24              1.0   
1         1  2022-02-01 00:38:22   2022-02-01 00:55:55              1.0   
2         1  2022-02-01 00:03:20   2022-02-01 00:26:59              1.0   
3         2  2022-02-01 00:08:00   2022-02-01 00:28:05              1.0   
4         2  2022-02-01 00:06:48   2022-02-01 00:33:07              1.0   
5         1  2022-02-01 00:57:23   2022-02-01 01:07:31              2.0   
6         1  2022-02-01 00:34:17   2022-02-01 01:01:32              1.0   
7         2  2022-02-01 00:03:26   2022-02-01 00:07:30              1.0   
8         2  2022-02-01 00:13:31   2022-02-01 00:31:41              1.0   
9         2  2022-02-01 00:52:11   2022-02-01 01:02:48              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           5.40         1.0                  N           138           252   
1           6.40